Run the best ADAbooster and the best GradientBooster

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier

Preprocess test data

In [19]:
df = pd.read_csv("test.csv")
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [20]:
passenger = df["PassengerId"]

In [3]:
# load encoders
import joblib

encoder = joblib.load("encoder.joblib")
scaler = joblib.load("scaler.joblib")

In [4]:
original = pd.read_csv("train.csv")

In [15]:
def unwrap_cabin(cabin):
    if pd.isna(cabin):
        return [None, None, None]
    else:
        parts = cabin.split('/')
        deck = parts[0]
        num = int(parts[1])
        side = parts[2] if len(parts) > 1 else None
        return [deck, num, side]
    
df['Deck'], df['Num'], df['Side'] = zip(*df['Cabin'].map(unwrap_cabin))
df['Missing_Cabin'] = df['Cabin'].isna().astype(int)
df.drop(columns=['Cabin'], inplace=True)

def one_hot_encode(df, columns):
    encoded_columns = encoder.transform(df[columns])

    encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(columns))
    df = df.join(encoded_df)
    return df.drop(columns, axis=1)

oh_columns = ['HomePlanet', 'Deck', 'Destination']
df = one_hot_encode(df, oh_columns)



df[['CryoSleep', 'VIP']] = df[['CryoSleep', 'VIP']].astype('Int64')
df['Side'] = df['Side'].map({'P': 1, 'S': 0})


df.drop(columns=['PassengerId', 'Name', 'Deck_None', 'Destination_nan', 'HomePlanet_nan'], inplace=True)

df = df.astype('float32')

def replace_nan_with_mean(df, columns):
    for col in columns:
        mean_value = original[col].mean()
        df[col].fillna(mean_value, inplace=True)
    return df

nan_columns = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CryoSleep', 'VIP']
df = replace_nan_with_mean(df, nan_columns)



df['Num'].fillna(0, inplace=True)
df['Side'].fillna(0, inplace=True)

columns_to_scale = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Num', 'Age']
df[columns_to_scale] = scaler.transform(df[columns_to_scale])

In [16]:
x = df.to_numpy()

In [ ]:
ada = joblib.load("AdaBoostClassifier.joblib")
preds = ada.predict(x)

final_pred = pd.DataFrame({"PassengerId": passenger, "Transported": preds.astype("bool")})
final_pred.to_csv("pred1.csv", index=False)

In [24]:
grad = joblib.load("GradientBoostingClassifier.joblib")
preds = grad.predict(x)

final_pred = pd.DataFrame({"PassengerId": passenger, "Transported": preds.astype("bool")})
final_pred.to_csv("pred2.csv", index=False)